In [ ]:
from ALLCools.mcds import MCDS
from ALLCools.plot import *
from ALLCools.integration import confusion_matrix_clustering

from wmb import cemba, aibs, broad, brain

import pandas as pd
import numpy as np
import anndata
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import wmb

In [ ]:
mc_annot = cemba.get_mc_annot()
rna_annot = aibs.get_smart_annot()

# plot level 1

In [ ]:
adata_merge = anndata.read_h5ad(f'../L1/Neuron/final_with_coords.h5ad')
rna_adata = adata_merge[adata_merge.obs['Modality'] == 'RNA'].copy()
mc_adata = adata_merge[adata_merge.obs['Modality'] == 'mC'].copy()

In [ ]:
rna_adata.obs['SubRegion'] = rna_annot['SubRegion'].to_pandas()
mc_adata.obs['SubRegion'] = mc_annot['SubRegion'].to_pandas()

# plot level 2

In [ ]:
def get_pre_data(integroup, category_key):
    #get adata
    adata_merge = anndata.read_h5ad(f'../{category_key}/ALL/{integroup}/final_with_coords.h5ad')
    rna_adata = adata_merge[adata_merge.obs['Modality'] == 'RNA'].copy()
    mc_adata = adata_merge[adata_merge.obs['Modality'] == 'mC'].copy()
    
    rna_meta = adata_merge.obs[adata_merge.obs['Modality'] == 'RNA'].copy()
    mc_meta = adata_merge.obs[adata_merge.obs['Modality'] == 'mC'].copy()
    
    #add L1 annot
    mc_adata.obs['L1_annot'] = mc_annot['L1_annot'].to_pandas()
    rna_adata.obs['L1_annot'] = rna_annot['L1'].to_pandas()
    
    #get integroup
    rna_integroup = pd.read_csv(f'../{category_key}/ALL/{integroup}/rna_integration_group.csv.gz', index_col = 'cell').squeeze()
    mc_integroup = pd.read_csv(f'../{category_key}/ALL/{integroup}/mc_integration_group.csv.gz', index_col = 'cell').squeeze()
    
    rna_adata.obs[f'{category_key}_InteGroup'] = rna_adata.obs.index.map(rna_integroup)
    rna_adata.obs[f'{category_key}_InteGroup'].value_counts()
    
    mc_adata.obs[f'{category_key}_InteGroup'] = mc_adata.obs.index.map(mc_integroup)
    mc_adata.obs[f'{category_key}_InteGroup'].value_counts()
    
    return rna_adata, mc_adata
    
        

In [ ]:
def plot_clustering(category_key):
    
    from ALLCools.plot.color import level_one_palette

    inte_group_palette = level_one_palette(
        pd.concat([rna_adata.obs[f'{category_key}_InteGroup'], mc_adata.obs[f'{category_key}_InteGroup']]), 
        palette='tab20'
    )
    
    fig, axes = plt.subplots(figsize=(10, 15), ncols=2, nrows=3, dpi=200)

    ax = axes[0, 0]
    categorical_scatter(ax=ax,
                        data=rna_adata,
                        coord_base='tsne',
                        hue=f'{category_key}_InteGroup',
                        text_anno=f'{category_key}_InteGroup',
                        palette=inte_group_palette,
                        max_points=None)
    ax.set(title='rna Inte. Group')

    ax = axes[0, 1]
    categorical_scatter(ax=ax,
                        data=mc_adata,
                        coord_base='tsne',
                        hue=f'{category_key}_InteGroup',
                        text_anno=f'{category_key}_InteGroup',
                        palette=inte_group_palette,
                        max_points=None)
    ax.set(title='mC Inte. Group')

    ax = axes[1, 0]
    categorical_scatter(ax=ax,
                        data=rna_adata,
                        coord_base='tsne',
                        palette='tab20',
                        hue='L3',
                        text_anno='L3',
                        max_points=None)
    ax.set(title=f'rna L3')
    ax.set(title=f'rna L3')


    ax = axes[1, 1]
    categorical_scatter(ax=ax,
                        data=mc_adata,
                        coord_base='tsne',
                        palette='tab20',
                        hue='L3',
                        text_anno='L3',
                        max_points=None)
    ax.set(title=f'mC L3')

    ax = axes[2, 0]
    categorical_scatter(ax=ax,
                        data=rna_adata,
                        coord_base='tsne',
                        hue='DissectionRegion',
                        text_anno='DissectionRegion',
                        palette='tab20',
                        max_points=None)
    ax.set(title='rna DissectionRegionp')

    ax = axes[2, 1]
    categorical_scatter(ax=ax,
                        data=mc_adata,
                        coord_base='tsne',
                        palette=wmb.brain.get_dissection_region_palette(region_type = 'CEMBA'),
                        hue='DissectionRegion',
                        text_anno='DissectionRegion',
                        max_points=None)
    ax.set(title=f'mC DissectionRegion')

In [ ]:
# by changing the category_key, can see all clustering results at each level
category_key = "L4"

In [ ]:
integroups = []
for i in pathlib.Path(f'../{category_key}/ALL').glob('InteGroup*'):
    integroups.append(str(i).split('/')[-1])

In [ ]:
with PdfPages(f'{category_key}_clusers.pdf') as pdf:
    for integroup in integroups:
        rna_adata, mc_adata = get_pre_data(integroup, category_key)
        plot_clustering(category_key)
        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()